### SageMaker  Endpoint 部署ChatGLM
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::432088571089:role/AmazonSageMaker-ExecutionRole-20210324T123126
sagemaker-us-east-1-432088571089


In [3]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatglm')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatglm')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-1-432088571089/chatglm/assets/model.tar.gz


### 设置模型推理参数
1. model_name: Huggingface diffusers models (not support single check point format)
2. model_args: diffuser StableDiffusionPipeline init arguments

In [11]:
model_name = None
entry_point = 'inference-chatglm.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [12]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

### 部署 SageMaker Endpoint
部署推理服务

In [13]:
endpoint_name = None
instance_type = 'ml.g4dn.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

--------!

### ChatGLM 测试


In [ ]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [16]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。
以下方法是一些可以尝试的来帮助在晚上入睡:

1. 创建一个舒适的睡眠环境:保持房间安静、凉爽、黑暗,并使用舒适的床垫和枕头。

2. 避免刺激性物质:避免饮用含咖啡因的饮料、饮酒或吸烟。也避免在睡前吃过多的食物或饮用含糖分的饮料。

3. 放松自己:尝试进行深呼吸、渐进性肌肉松弛或冥想等放松技巧。也可以听一些轻柔的音乐或进行瑜伽练习来放松自己。

4. 制定规律的睡眠时间表:保持每天相同的起床时间和睡眠时间,让身体适应这个规律。

5. 避免在床上使用电子设备:避免使用手机、平板电脑或电视等电子设备,因为它们会刺激大脑,使人难以入睡。

6. 练习睡前放松技巧:尝试进行冥想或放松瑜伽,帮助身体和大脑放松,准备进入睡眠状态。

如果这些方法不能帮助入睡,建议咨询医生或睡眠专家,了解是否有其他问题或需要进一步的帮助。


In [ ]:
inputs= {
    "ask": "列出一些年夜饭好意头的菜肴以及其寓意!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一篇人工智能课程的教案，1000字"

}

response = predictor.predict(inputs)
print(response["answer"])

In [22]:
inputs= {
    "ask": "怎么修改huggingface transformers的model cache位置"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "用python3写出快速排序代码"

}

response = predictor.predict(inputs)
print(response["answer"])

要修改Hugging Face Transformers的模型缓存位置,您可以按照以下步骤进行操作:

1. 打开Hugging Face Transformers的配置文件(例如`transformers.config`)。

2. 在配置文件中找到` cache.size`和` cache.dir`参数。`cache.size`指定了模型缓存的大小,而`cache.dir`指定了模型缓存的位置。

3. 修改`cache.dir`参数,为您想要缓存模型的位置创建一个目录。例如,如果您要将模型缓存放在`/path/to/cache/`目录中,则可以将`cache.dir`设置为`/path/to/cache/`。

4. 保存并重新加载模型。

5. 在加载模型时,您应该会看到模型缓存的位置改变了。

例如,假设您要将模型缓存放在`/path/to/cache/my_model_dir`目录中,您可以按照以下步骤修改配置文件:

```
cache.dir = /path/to/cache/my_model_dir
```

请注意,`cache.dir`参数必须是一个目录,并且目录名称应该与模型缓存的位置相同。
以下是使用 Python 3 编写的快速排序代码示例:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        left = []
        right = []
        for i in range(1, len(arr)):
            if arr[i] < pivot:
                left.append(arr[i])
            else:
                right.append(arr[i])
        return quicksort(left) + [pivot] + quicksort(right)
```

这个函数接受一个列表 `arr` 作为输入,并返回一个新的已排序的列表。如果输入列表的长度小于等于 1,则直接返回列表本身。否则,找到第一个元素 `pivot`,并将整个列表分成两

In [21]:
#我们来查看一下刚部署的这个ChatGLM模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

'pytorch-inference-2023-03-15-07-58-44-863'

利用已经在SageMaker real time inference endpoint部署的ChatGLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [18]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下ChatGLM针对单个问题的回答

In [19]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

阿根廷,梅西。


单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [20]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

用户：你好！
小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。
用户：今天天气怎么养
小元：抱歉,我不太明白您的问题。能否提供更多上下文或信息,让我更好地理解您的问题?谢谢!
用户：今天天气怎么？
小元：抱歉,作为一个语言模型,我没有实时的天气信息。请查看当地的天气预报或使用天气应用程序来获取当前和未来几天的天气情况。
用户：如何下一篇不错的学术论文
小元：下一篇不错的学术论文需要以下几个步骤:

1. 阅读原始论文:阅读前一篇学术论文是了解当前研究领域和最新研究趋势的关键。因此,应该仔细阅读原始论文,了解作者的研究内容和结论,以及他们的方法、数据和实践。

2. 确定目标论文:在阅读原始论文后,需要确定自己想要阅读的下一篇学术论文,以便更好地了解该领域的研究进展和趋势。目标论文应该与自己的研究方向相关,具有创新性和实用性,同时也需要与原始论文进行一定的比较和联系。

3. 查找相关文献:在阅读目标论文之前,需要了解该领域的先前研究,以更好地理解作者的研究内容和结论。可以通过图书馆、学术数据库、学术论坛等途径查找相关的文献,以便更好地了解该领域的研究现状和趋势。

4. 关注论文质量:一篇好的学术论文应该具有清晰、简洁、准确的表达,合理的结构和逻辑推理,丰富的数据和实证分析,以及清晰、简洁的摘要和结论。因此,在阅读学术论文时,应该注意论文的质量和结构,以便更好地理解和吸收作者的研究成果。

5. 与同行交流:与同行交流是了解研究领域和最新研究趋势的重要途径。可以通过学术会议、学术论坛、电子邮件、博客等途径与同行交流,分享自己的想法和研究成果,并从他人的研究成果中学习和借鉴。
用户：quit


多轮对话模拟：我们这里来测试一下ChatGLM的多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导ChatGLM，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉ChatGLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()